In [1]:
import os
print(os.getcwd())

/home/jovyan/59min/model


In [2]:
#!pip install torch==2.5.0+cu121 torchaudio==2.5.0+cu121 torchvision==0.20.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install transformers
#!pip install openpyxl
#!pip install rank-bm25

In [3]:
from transformers import BertTokenizer
import random
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import openpyxl
import ast
import torch.optim as optim

# 데이터 준비

## Dataset 1: 키워드화 된 데이터셋 가져오기

In [4]:
# 엑셀 파일 불러오기
file_path_full = '/home/jovyan/59min/Bert4Rec/Performance_evaluation_dataset/Korea_full_sentence.xlsx'  # 파일 경로

df_full = pd.read_excel(file_path_full)

In [5]:
df_full = df_full.rename(columns={'keywords':'label'})

In [6]:
# 엑셀 파일 불러오기
file_path_user_sequence= '/home/jovyan/59min/model/user_sequence_ko.xlsx'  # 파일 경로
df_seq = pd.read_excel(file_path_user_sequence)

df_seq = df_seq.drop(columns=['Unnamed: 0'])

In [7]:
df_full

,선택 이유,베뉴 이름
0,홀 내부- 단독홀로 타 하객들과 섞이지 않고 비교적 넉넉한 시간동안 이용할 수 있다...,빌라드지디 수서
1,저희 커플이 홀 선택 시 고려했던 점은 예쁜 홀 + 맛있는 식사였습니다.(1) 홀 ...,더링크호텔 가든홀
2,"인천에서 가장 많이 한다는 네임밸류, 검증된 식사 퀄리티 결정적으로 신부가 홀에 마...",메리빌리아 셀레스메리홀
3,밝은홀의 장점과 어두운 홀의 장점 모두 아우르고 있는것 같아서 선택! ( 자세한 사...,노블발렌티 삼성점
4,저희가 보증인원 100-150명이 가능한 홀을 찾다보니 옵션이 많이 않았습니다.소규...,국방 컨벤션
...,...,...
292,사랑스러운 분위기의 홀이 제일 먼저 눈에 들어왔어요- 하객의자가 예쁘고 숲속느낌이 ...,수원 파티움 하우스
293,"몇 번 하객으로 왔던 경험 상 무난하다고 생각했고, 식장과 음식 퀄리티는 보장된다고...",신도림 라마다 5층 세인트그레이스홀
294,저희는 우선 하객들에게 식사를 대접하는 예식을 가장 우선으로 꼽아서 플래너님에게도 ...,웨딩여율리
295,"강남에 위치한 채플형 웨딩홀을 원했기 때문에 아펠가모 2곳 (선릉, 반포) + 더채...",더채플앳논현 라메르홀


In [8]:
df_seq

,user_id,list,keyword
0,0,"['국방 컨벤션', '빌라드지디 수서', '라비두스', '시그니엘', '신도림 더 ...",예쁜 홀
1,1,"['더링크호텔', '더컨벤션 신사']",예쁜 홀
2,2,"['더링크호텔 가든홀', '더마레보호텔&웨딩', '아펠가모 선릉']",예쁜 홀
3,3,"['노블발렌티 삼성', '양재 엘블레스', '잠실 아펠가모 웨딩홀', '구로 지타워...",예쁜 홀
4,4,"['신도림 테크노마트 웨딩시티', '서울대 이라운지', '♥ 노블발렌티 삼성점']",예쁜 홀
...,...,...,...
645,645,"['노블발렌티 삼성역점', '노블발렌티 삼성']",신부 대기
646,646,"['국방 컨벤션', '루벨 강동']",신부 대기
647,647,"['크레스트72 글라스홀', '양재 엘타워 그랜드홀']",신부 대기
648,648,"['양재 엘타워 그랜드홀', '노블발렌티 삼성', '더파티움 안양(구 아르떼)', ...",신부 대기


# Bert4Rec 학습

In [9]:
# # venue_data 생성
# venue_data = {}
# for _, row in df_full.iterrows():
#     venue_data[row['베뉴 이름']] = {
#         "선택 이유": row['선택 이유']
#     }

# user_sequences = df_seq['list']


In [10]:
class BERT4RecDataset(Dataset):
    def __init__(self, data, max_seq_len=10, mask_prob=0.15):
        self.user_sequences = data['list'].apply(ast.literal_eval)  # 시퀀스를 리스트 형태로 변환
        self.mask_prob = mask_prob
        self.max_seq_len = max_seq_len
        self.venue_names = list(set([venue for seq in self.user_sequences for venue in seq]))  # 모든 venue 리스트화
        self.vocab_size = len(self.venue_names)
        self.data = self.prepare_data()

    def prepare_data(self):
        data = []
        for seq in self.user_sequences:
            indexed_seq = []
            for item in seq:
                if item in self.venue_names:
                    indexed_seq.append(self.venue_names.index(item))
                else:
                    print(f"Warning: {item} is not in venue_names, replacing with [UNK]")
                    indexed_seq.append(self.vocab_size - 1)  # [UNK] 토큰으로 대체

            # 패딩 또는 잘라내기
            if len(indexed_seq) < self.max_seq_len:
                indexed_seq = [0] * (self.max_seq_len - len(indexed_seq)) + indexed_seq  # 패딩
            else:
                indexed_seq = indexed_seq[:self.max_seq_len]
            data.append(indexed_seq)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.data[idx]
        input_ids = torch.tensor(seq, dtype=torch.long)
        labels = torch.tensor(seq, dtype=torch.long)
        masked_input = input_ids.clone()
        mask = torch.zeros_like(input_ids, dtype=torch.bool)

        # 마스킹 로직
        for i in range(len(seq)):
            if random.random() < self.mask_prob:
                mask[i] = True
                prob = random.random()
                if prob < 0.8:
                    masked_input[i] = self.vocab_size - 1  # 마스킹 토큰 추가
                elif prob < 0.9:
                    masked_input[i] = random.choice(list(range(self.vocab_size)))  # 랜덤하게 다른 토큰 선택

        return masked_input, labels, mask


In [11]:
# BERT4Rec 모델 정의 (간단화된 BERT 구조)
class BERT4RecModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, num_heads=4, num_layers=2, max_seq_len=10, dropout=0.1):
        super(BERT4RecModel, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.position_embedding = nn.Embedding(max_seq_len, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, input_ids):
        positions = torch.arange(0, input_ids.size(1)).unsqueeze(0).to(input_ids.device)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        logits = self.fc(x)
        return logits

In [12]:
# 데이터셋 로드 및 DataLoader 설정
dataset = BERT4RecDataset(df_seq)  # user_sequences_df: 유저 시퀀스가 포함된 데이터프레임
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 모델 초기화
vocab_size = len(dataset.venue_names)
model = BERT4RecModel(vocab_size=vocab_size)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

/home/jovyan/.venv/torch2.3.0-py3.11-cuda12.1/lib/python3.11/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [13]:
# 모델 학습 루프
model.train()
for epoch in range(15):  # 예시로 15번의 epoch
    total_loss = 0
    for batch in dataloader:
        input_ids, labels, masks = batch
        optimizer.zero_grad()
        logits = model(input_ids)
        logits = logits.view(-1, vocab_size)
        labels = labels.view(-1)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")

Epoch 1, Loss: 3.2894
Epoch 2, Loss: 1.5830
Epoch 3, Loss: 1.0654
Epoch 4, Loss: 0.9093
Epoch 5, Loss: 0.8349
Epoch 6, Loss: 0.8007
Epoch 7, Loss: 0.7909
Epoch 8, Loss: 0.7819
Epoch 9, Loss: 0.7060
Epoch 10, Loss: 0.7167
Epoch 11, Loss: 0.7416
Epoch 12, Loss: 0.6707
Epoch 13, Loss: 0.7083
Epoch 14, Loss: 0.6492
Epoch 15, Loss: 0.6984


In [14]:
# 모델 저장 경로 설정
model_save_path = 'bert4rec_model_ko.pth'


In [15]:
# 모델의 가중치 저장
torch.save(model.state_dict(), model_save_path)
print("Model saved to", model_save_path)

Model saved to bert4rec_model_ko.pth
